In [1]:
import pandas as pd
import numpy as np

In [119]:
keri = pd.read_csv("/storage/group/ibb3/default/darpa/Keri_Metadata.csv")

In [120]:
rich1 = pd.read_csv("/storage/group/ibb3/default/darpa/2023-03-21_psuMeta.csv")

In [121]:
rich2 = pd.read_csv("/storage/group/ibb3/default/darpa/cruiseCbassMetadata.csv")

In [122]:
rich2 = rich2[rich2["genetNumber"].notna()]
rich2["genetNumber"] = rich2["genetNumber"].astype(int)

In [123]:
rich1 = rich1.merge(rich2[["genetNumber", "registryNumber", "Region", "Collection Site"]], 
            on = "genetNumber", how = "left")

In [124]:
keri = keri[keri["region"] == "Florida"][keri["reef"] != "2017 Spawn"][keri["reef"] != "2020 Spawn"][keri["field_call"] == "palmata"].reset_index(drop=True)

/tmp/ipykernel_1151386/117862151.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  keri = keri[keri["region"] == "Florida"][keri["reef"] != "2017 Spawn"][keri["reef"] != "2020 Spawn"][keri["field_call"] == "palmata"].reset_index(drop=True)


In [125]:
keri.columns = ['#user_specimen_id', 'field_call', 'bcoral_genet_id', 'bsym_genet_id',
       'Coral Mlg Clonal ID', 'reef', 'region', 'latitude', 'longitude',
       'geographic_origin', 'colony_location', 'depth', 'disease_resist',
       'bleach_resist', 'mortality', 'tle', 'spawning', 'collector_last_name',
       'collector_first_name', 'organization', 'collection_date', 'email',
       'seq_facility', 'array_version', 'public', 'public_after_date',
       'sperm_motility', 'healing_time', 'dna_extraction_method',
       'dna_concentration', 'registry_id', 'result_folder_name',
       'plate_barcode',
       'notes']

In [126]:
keri = keri[['#user_specimen_id', 'Coral Mlg Clonal ID', 'reef', 'latitude', 'longitude', 'spawning', 'notes']]

In [127]:
keri.columns = ["genetNumber", "clonal_id_correct", "Collection Site", "Latitude", "Longitude", "spawning", "notes"]

In [128]:
keri["Provider"] = "Keri"

In [129]:
rich1["Provider"] = "Rich"

In [145]:
combined = pd.concat([rich1, keri]).reset_index(drop=True)

In [146]:
combined.loc[combined['Latitude'] > 24.95, "Region"] = "Upper Keys"
combined.loc[(combined['Latitude'] < 24.95) & (combined['Latitude'] > 24.547), "Region"] = "Middle Keys"
combined.loc[combined['Latitude'] < 24.547, "Region"] = "Lower Keys"

In [147]:
combined.loc[combined['genetNumber'] == "APAL 15", "galaxyCall"] = "Apal-009"
combined.loc[combined['genetNumber'] == "APAL 10", "galaxyCall"] = "Apal-154"
combined.loc[combined['genetNumber'] == "APAL 12", "galaxyCall"] = "Apal-161"
combined.loc[combined['genetNumber'] == "APAL 16", "galaxyCall"] = "M3APBNP1"
combined.loc[combined['genetNumber'] == "APAL 17", "galaxyCall"] = "AMAP5"
combined.loc[combined['genetNumber'] == "APAL 18", "galaxyCall"] = "AMAP1"
combined.loc[combined['genetNumber'] == "APAL 19", "galaxyCall"] = "BALLBUOY0"
combined.loc[combined['genetNumber'] == "APAL 20", "galaxyCall"] = "FWCAP1"
combined.loc[combined['genetNumber'] == "APAL 21", "galaxyCall"] = "BALL PAUL"
combined.loc[combined['genetNumber'] == "APAL 22", "galaxyCall"] = "BREWAP"
combined.loc[combined['genetNumber'] == "APAL 23", "galaxyCall"] = "BREWAPWEST"
combined.loc[combined['genetNumber'] == "APAL 24", "galaxyCall"] = "Apal-018"

Some of these samples were already run by other people (or will be run), so we need to exclude them:

In [148]:
combined = (combined
 [~combined["galaxyCall"].isin(["Apal-005", "Apal-016", "Apal-044", "Apal-047", "Apal-074", "Apal-090"])]
 [~combined["clonal_id_correct"].isin(['HG1714', 'HG0799', 'HG1738', 'HG1734', 'HG0822', 'HG1714'])]).reset_index(drop=True)

/tmp/ipykernel_1151386/3905822483.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  combined = (combined


In [149]:
combined = (combined
 [~combined["clonal_id_correct"].isin(['HG0600', 'HG1092', 'HG1042', 'HG0544', 'HG0543', 'HG0548',
                                       "HG0611", "HG0616", "HG0606", "HG1093", "HG1118", "HG1045",
                                       "HG1129", "HG1071", "HG1077", "HG1120", "HG1090", "HG1096",
                                       "HG1068", "HG1106"])]).reset_index(drop=True)

In [151]:
combined = combined.sort_values(["clonal_id_correct", "Provider", "Nursery"]).reset_index(drop=True)

In [152]:
combined = combined[(~combined["clonal_id_correct"].duplicated()) | 
                    (combined["clonal_id_correct"].isnull())].reset_index(drop=True)

In [153]:
combined = combined[(~combined["galaxyCall"].duplicated()) | 
                    (combined["galaxyCall"].isnull())].reset_index(drop=True)

In [161]:
combined = combined[combined["Region"].notna()].sort_values(["Latitude", "Region"]).reset_index(drop=True)

In [163]:
combined.columns

Index(['genetNumber', 'Origin', 'cbassRun', 'Nursery', 'galaxyCall', 'Genet',
       'ed50', 'std.error', 'n', 'standard', 'totSym', 'propD', 'dominant',
       'Latitude', 'Longitude', 'top', 'Affy_ID', 'clonal_id_correct',
       'Species_FieldCall', 'registryNumber', 'Region', 'Collection Site',
       'Provider', 'spawning', 'notes'],
      dtype='object')

In [187]:
output_metadata = combined[['genetNumber', 'galaxyCall', 'Genet',
       'Collection Site', 'Latitude', 'Longitude', 'Region', 'Nursery', 'clonal_id_correct', 
       'Provider', 'spawning', 'notes']]

In [190]:
output_metadata.columns = ['SampleProviderTubeID', 
                           'Galaxy Name', 
                           'Genet', 
                           'Original Collection Site', 
                           'Latitude', 'Longitude', 
                           'Region', 
                           'Nursery', 
                           'Coral Mlg Clonal ID', 
                           'SampleProvider',
                           'Spawning', "Notes"]

In [191]:
output_metadata.to_csv("/storage/group/ibb3/default/darpa/filtered_DARPA_metadata.csv")

In [195]:
output_metadata["Coral Mlg Clonal ID"].value_counts(dropna=False)

NaN       28
HG1185     1
HG1736     1
HG1653     1
HG0837     1
          ..
HG1633     1
HG0816     1
HG0205     1
HG0828     1
HG1737     1
Name: Coral Mlg Clonal ID, Length: 71, dtype: int64

In [3]:
tmp = pd.read_csv("/storage/group/ibb3/default/darpa/db_upload.csv")

In [4]:
meta = pd.read_csv("/storage/group/ibb3/default/darpa/filtered_DARPA_metadata.csv")

In [8]:
tmp = tmp[["Database ID", "Field ID"]]
tmp.columns = ["Database ID", "SampleProviderTubeID"]

In [10]:
meta = meta.merge(tmp, how = "left", on = 'SampleProviderTubeID')

In [15]:
meta["Database ID"][93] = 13414

/tmp/ipykernel_207503/1611924406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta["Database ID"][93] = 13414


In [17]:
meta = meta[meta["Database ID"].notna()].reset_index(drop=True)

In [18]:
meta["Database ID"] = meta["Database ID"].astype(int)

In [21]:
meta[["Huck_Sub_ID", "Database ID"]].set_index("Huck_Sub_ID").to_csv("/storage/group/ibb3/default/darpa/huck_sub_id.csv")